In [55]:
from kalah import *
# from pebblereaper import *
from extrakalah import *
depth = 6
weights = [0.51, 0.01, 0.34, 0.1]
h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds]
full_h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds,capture_defense]
alt_h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds,capture_available]

class Jogador():
    def __init__(self, nome, fun):
        self.nome = nome
        self.fun = fun
    def display(self):
        print(self.nome+" ")
        
class JogadorAlfaBeta(Jogador):
    def __init__(self, nome, depth,fun_eval):
        self.nome = nome
        self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)


##########  para ser independente dos jogos deveria devolver um método em string ou um atributo
def joga11(game, jog1, jog2,verbose=False):
    ### jog1 e jog2 são jogadores com funções que dado um estado do jogo devolvem a jogada que escolheram
    ### devolve o par de jogadores, a lista de jogadas e o resultado
    estado=game.initial
    proxjog = jog1
    lista_jogadas=[]
    lance = 0
    while not game.terminal_test(estado):
        if verbose:
            print('----------   LANCE:',lance)
            game.display(estado)
        jogada = proxjog.fun(game, estado) # state produced
        if verbose:
            print('JOGADA=',jogada)
            print("pit_total:",pit_total(estado,proxjog))
            print("side_total: ", side_total(estado,proxjog))
            print("chance_to_clear_right_most:",chance_to_clear_right_most(estado,proxjog))
            print("congregate_seeds: ", congregate_seeds(estado,proxjog))
            print("heuristic total:",linearPond(estado,proxjog,weights,h_list,100))
        estado=game.result(estado,jogada)
        lista_jogadas.append(jogada)
        proxjog = jog2 if proxjog == jog1 else jog1
        lance+=1
    #p jogou e ganhou
    util=game.utility(estado,0)
    if util == 1:
        resultado=jog1.nome
    elif util== -1:
        resultado = jog2.nome
    else:
        resultado='Empate'
    return ((jog1.nome,jog2.nome),lista_jogadas,resultado)


In [62]:
'''
Pebbles "The Reaper" Snatcher 
     [____]   
     ]()()[   
   ___\__/___ 
  |__|    |__|
   |_|_/\_|_| 
   | | __ | | 
   |_|[::]|_| 
   \_|_||_|_/ 
     |_||_| 
    _|_||_| jro
   |___||___| 
'''

# from jogos import *
# from testing import *
from kalah import KalahState
# from jogos import alphabeta_cutoff_search_new
# class Jogador():
#     def __init__(self, nome, fun):
#         self.nome = nome
#         self.fun = fun
#     def display(self):
#         print(self.nome+" ")
        
# class JogadorAlfaBeta(Jogador):
#     def __init__(self, nome, depth,fun_eval):
#         self.nome = nome
#         self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)


# PebblePlayer = JogadorAlfaBeta("PebbleReaper",2,)

# # O primeiro é o 0 e o segundo é o 1 (para jogos de dois jogadores).
# def linearPond(estado,jogador,pesos,funcoes,win):
#     """Função que pega no estado e no jogador e se fim do jogo devolve win*utilidade (* 1 ou -1, dependendo do jogador)
#     senão devolve a combinação linear de pesos e features."""
#     if estado.is_game_over():
#         aux = estado.result()
#         return aux*win if jogador == 0 else aux*-win
#     # print("pit_total:",pit_total(estado,proxjog))
#     # print("side_total: ", side_total(estado,proxjog))
#     # print("chance_to_clear_right_most:",chance_to_clear_right_most(estado,proxjog))
#     # print("congregate_seeds: ", congregate_seeds(estado,proxjog))
#     return sum([p*f(estado,jogador) for (p,f) in zip(pesos,funcoes)])


# class JogadorLinearPond(JogadorAlfaBeta):
#     """SubClasse dos jogadores alfabeta que usa o combinador linear como função de avaliação.
#     Recebe para além do nome e da profundidade, a lista de pesos e das funções-features, criando fun_eval"""
#     def __init__(self, nome, depth,weights,features,win_value):
#         self.nome = nome
#         fun_eval=lambda estado,jogador: linearPond(estado,jogador,weights,features,win_value)
#         self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)

def func_42(estado,jogador):
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == 0 else aux*-100
    
    # if random.randrange(0,10) <= 3:
    ###defense strat###
    weights = [0.4132746969431487, 0.004000018652774955, 0.09436108536199529, 0.0566795623905725, 0.029663753065053162, 0.4020208835864555]
    functions = [pit_total_42,side_total_42,chance_to_clear_right_most_42,congregate_seeds_42,capture_available_42,chaining_42]
    # else:
    ###base###
    # weights = [0.51, 0.01, 0.34, 0.1]
    # functions = [pit_total_42,side_total_42,chance_to_clear_right_most_42,congregate_seeds_42]
    
    return sum([p*f(estado,jogador) for (p,f) in zip(weights,functions)])


'''
REF: https://www.mdpi.com/2227-9709/7/3/34
ok    H1: Collect as many seeds in one pit as possible. Prioritize Left pits over right pits
ok    H2: Collect as many seeds on the player’s side.
n/a    H3: Reach as many moves as possible to select.
ok    H4: Maximize the number of seeds in the player’s kalaha.
ok    H5: Clear the rightmost pit (i.e., that with number 1).
ok    H6: Minimize the number of seeds in the opponent’s kalaha. Prevent seeds in opposition's kalaha.
'''
# satisfies H4: Maximize the number of seeds in the player’s kalaha.
def pit_total_42(state:KalahState,player):
    player_pit = state.state[6] if player == state.SOUTH else state.state[13]
    opponent_pit = state.state[13] if player == state.SOUTH else state.state[6] 
    return player_pit-opponent_pit

# satisfies H2:  Collect as many seeds on the player’s side.
def side_total_42(state:KalahState,player):
    player_side_total = sum(state.state[:6]) if player == state.SOUTH else sum(state.state[7:13]) 
    opponent_side_total = sum(state.state[7:13]) if player == state.SOUTH else sum(state.state[:6])
    return player_side_total-opponent_side_total

# satsifies H5: Clear the rightmost pit (i.e., that with number 1).
def chance_to_clear_right_most_42(state:KalahState,player):
    north_right_empty = 1 if state.state[12] == 0 else 0
    south_right_empty = 1 if state.state[5] == 0 else 0
    return south_right_empty - north_right_empty if player == state.SOUTH else north_right_empty - south_right_empty

# satsifies H1: Collect as many seeds in one pit as possible
# encourages having more seeds on the pits closer to the left hand side
def congregate_seeds_42(state:KalahState,player):
    weights = {0:1,
               1:0.8,
               2:0.6,
               3:0.4,
               4:0.2}
    
    south_result = 0
    for i in range(5):
        pit_weight = state.state[i]*weights[i]
        if  pit_weight > south_result:
            south_result = pit_weight
    north_result = 0
    for i in range(7,12):
        pit_weight = state.state[i]*weights[i-7]
        if  pit_weight > north_result:
            north_result = pit_weight
    return south_result - north_result if player == state.SOUTH else north_result - south_result


# Not used.
# This supposes a given state isn't played and could have potential captures
def capture_available_42(state:KalahState,player):
    board = state.state

    south_max_capture = 0
    south_empties = [i for i in range(6) if board[i]==0]
    south_move_final_index = [(i+board[i])%14 if i+board[i]<13 else (i+board[i])%13 for i in range(6) if board[i] != 0 and board[i]<=13]
    south_possible_captures = [value for value in south_empties if value in south_move_final_index]

    if len(south_possible_captures) != 0:
        for i in south_possible_captures:
            if 1+board[i+7] > south_max_capture:
                south_max_capture = 1+board[i+7]
    
    north_max_capture = 0
    north_empties = [i for i in range(7,13) if board[i]==0]
    north_move_final_index = [(i+board[i])%14 if i+board[i]-7<13 else ((i+board[i])%14)+1 for i in range(7,13) if board[i] != 0 and board[i]<=13]
    north_possible_captures = [value for value in north_empties if value in north_move_final_index]

    if len(north_possible_captures) != 0:
        for i in north_possible_captures:
            if 1+board[i-7] > north_max_capture:
                north_max_capture = 1+board[i-7]

    return south_max_capture - north_max_capture if player == state.SOUTH else north_max_capture - south_max_capture

# Check if any chaining can be used.
def chaining_42(state:KalahState,player):
    if(state.pass_turn == True):
        return 1
    return 0

# Detect possible captures from oppositions on the next move.
# Heuristics are taken away for every weak link
def capture_defense_42(state:KalahState,player):
    # for every capture blocked, the value is like, the number of seed they dont win
    board = state.state
    
    south_max_capture = 0
    south_empties = [i for i in range(6) if board[i]==0]
    south_move_final_index = [(i+board[i])%14 if i+board[i]<13 else (i+board[i])%13 for i in range(6) if board[i] != 0 and board[i]<=13]
    south_possible_captures = [value for value in south_empties if value in south_move_final_index]

    if len(south_possible_captures) != 0:
        for i in south_possible_captures:
            if 1+board[i+7] > south_max_capture:
                south_max_capture = 1+board[i+7] #figure out the best capture they could do
    
    north_max_capture = 0
    north_empties = [i for i in range(7,13) if board[i]==0]
    north_move_final_index = [(i+board[i])%14 if i+board[i]-7<13 else ((i+board[i])%14)+1 for i in range(7,13) if board[i] != 0 and board[i]<=13]
    north_possible_captures = [value for value in north_empties if value in north_move_final_index]

    if len(north_possible_captures) != 0:
        for i in north_possible_captures:
            if 1+board[i-7] > north_max_capture:
                north_max_capture = 1+board[i-7]

    if player == state.SOUTH:
        return pit_total_42(state,player) - north_max_capture*(-1)
    else:
        return pit_total_42(state,player) - south_max_capture*(-1)

## Create Chapiteau

In [57]:
def toes(state,player):
    board = state.state
    if state.is_game_over():
        aux = state.result()
        return aux*100 if player == state.SOUTH else aux*-100 
    
    own_seeds = sum(board[:7]) if player == 0 else sum(board[7:14])
    opponent_seeds = sum(board[7:14]) if player == 0 else sum(board[:7])
    score = own_seeds - opponent_seeds
    return score

toe1 = JogadorAlfaBeta("chapiteau",depth,toes)
# toe2 = JogadorAlfaBeta("heehoo",6,toes)



## Create Random Player

In [58]:
def f_caos_intel(estado,jogador):
    """Quando é terminal: +100 para vitória, -100 para a derrota e 0 para o empate.
       Quando o tabuleiro é não terminal devolve 0, o que quer dizer que como o minimax baralha as acções, será random"""
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == estado.SOUTH else aux*-100
    return 0

el_caos_int6=JogadorAlfaBeta("El Caos Inteligente 6",depth,f_caos_intel)

# Pebbles!

In [63]:
# import copy
# pebs = JogadorAlfaBeta('Pebbles',depth,Jogador_42)
# nome, depth,weights,features,win_value
h_list = [pit_total,side_total,chance_to_clear_right_most,congregate_seeds]
pebs =JogadorLinearPond('Pebbles',depth,weights,h_list,100)

#
# defense focused
#

# pebs2: more ags trained against chapiteau
# weights2 = copy.copy(weights)
# weights2.append(1)
weights2 = [0.74, 0.01, 0.09, 0.06, 0.15]
pebs2 = JogadorLinearPond('Pebbles2-defense-focused',depth,weights2,full_h_list,100)

# depth  8 gen 7
# weights3 = [0.368, 0.0, 0.242,0.152,0.237] #[0.74, 0.01, 0.09, 0.06, 0.15]
# pebs3 = JogadorLinearPond('Pebbles3-defense-focused',depth,weights3,full_h_list,100)

weights4 =  [0.53, 0.01, 0.03, 0.03, 0.04]
pebs4 = JogadorLinearPond('Pebbles4-defense-focused',depth,weights4,full_h_list,100)



# [0.74, 0.01, 0.09, 0.06, 0.15], [0.53, 0.01, 0.03, 0.03, 0.04], [0.8386724534373374, 0.0047944633947019175, 0.0, 0.15653308316796063, 0.0]
# capture focused
#
weights2b = [0.74, 0.01, 0.09, 0.06, 0.15]
pebs2b = JogadorLinearPond('Pebbles2-capture-focused',depth,weights2b,alt_h_list,100)

# weights3b = [0.368, 0.0, 0.242,0.152,0.237] #[0.74, 0.01, 0.09, 0.06, 0.15]
# pebs3b = JogadorLinearPond('Pebbles3-capture-focused',depth,weights3b,alt_h_list,100)

weights4b =  [0.53, 0.01, 0.03, 0.03, 0.04]
pebs4b = JogadorLinearPond('Pebbles4-capture-focused',depth,weights4b,alt_h_list,100)

weightsbc = [0.8386724534373374, 0.0047944633947019175, 0.0, 0.15653308316796063, 0.0]
pebs_bc = JogadorLinearPond('Pebbles_BestCapture',depth,weightsbc,alt_h_list,100)


weightsbd = [0.2586993457142682, 0.010680145858320323, 0.6757607019062054, 0.00742569201034956, 0.04743411451085642]
pebs_bd = JogadorLinearPond('Pebbles_BestDefense',depth,weightsbd,full_h_list,100)


# best_prime = [0.51, 0.01, 0.34, 0.1]

pebreaper = JogadorAlfaBeta("PebbleReaper",depth,func_42)


# One game

# N Match Pairs

In [42]:
scores={'Vitoria': 3, 'Empate': 1}

def traduzPontos(tabela):
    tabelaScore={}
    empates=tabela['Empate']
    for x in tabela:
        if x != 'Empate':
            tabelaScore[x]=scores['Vitoria']*tabela[x]+empates
    return tabelaScore

def getPoints(tabela, jogador):
    empates=tabela['Empate']
    total=0
    for x in tabela:
        total += scores['Vitoria']*tabela[x]+empates
    player_score = scores['Vitoria']*tabela[jogador]+empates
    return player_score/total

#ret_jog_index == 0 or 1 ONLY
def jogaNpares_GA(jogo,n,jog1,jog2,ret_jog_index):
    tabelaPrim={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabelaSeg={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabela={}
    print(' ', end='', flush=True)
    for _ in tqdm(range(n), desc= "jogaNpares_GA range %d"%jog2.nome,leave=False):
        _,_,vencedor=joga11(jogo,jog1,jog2)
        tabelaPrim[vencedor]+=1
        _,_,vencedor=joga11(jogo,jog2,jog1)
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    ret_jog = jog1.nome if ret_jog_index == 0 else jog2.nome
    return getPoints(tabela,ret_jog) # this is needed for GA

def jogaNpares(jogo,n,jog1,jog2):
    tabelaPrim={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabelaSeg={jog1.nome:0, jog2.nome:0, 'Empate':0}
    tabela={}
    for _ in range(n):
        _,_,vencedor=joga11(jogo,jog1,jog2)
        tabelaPrim[vencedor]+=1
        _,_,vencedor=joga11(jogo,jog2,jog1)
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    # ret_jog = jog1.nome if ret_jog_index == 0 else jog2.nome
    # return getPoients(tabela,ret_jog) # this is needed for GA
    return tabelaPrim,tabelaSeg,tabela,traduzPontos(tabela) # this gives better data for individual npare runs

# SUPER IMPORTANT: if you want to see the full table using jogaNpares rather than the numerical win rate for GA, comment/uncomment the last two lines above

In [15]:
# jogaNpares()

# # jogo=Kalah(0)
#n jogaNpares(jogo,300,el_caos,el_caos_int6)
# jogo.result()

# _,_,res=joga11(jogo,pebs,pebs2,True)
# jogaNpares(jogo,30,toe1,pebs)
# _,_,_,res=jogaNpares(jogo,10,pebs,pebs_bc)
# jogaNpares(jogo,30,toe1,pebs4)
# print(res)


{'Pebbles': 24, 'Pebbles_BestCapture': 36}


# SearchPlus Genetic Algorithm

In [ ]:
# from searchPlus import genetic_algorithm_custom
# import random

# # for steps in 0.01, use 2. 
# DEPTH = depth 
# PRECISION = 10
# HEURISTICS = alt_h_list
# N_WEIGHTS = 5
# TRAINING_ENEMY = pebs4
# N_GAMES = 10
# N_GENERATION = 1
# PMUT = 0.1
# # generates n weights that adds up to 1, precision see above
# # e.g. [0.7, 0.12, 0.12, 0.06]
# def generate_population(n,precision):
#     total = pow(10,precision)
#     dividers = random.sample(range(1, total), n - 1)
#     return [(a - b)/total for a, b in zip(dividers + [total], [0] + dividers)]

# # generates a pool of potential changes from 0-1
# # a list that goes [0.0, 0.01 ... 1] essentially. based on precision
# def generate_pool(precision):
#     return [round(x*pow(10,-precision),precision) for x in range(pow(10,precision))]
    
# # generates 8 "gene" as an initial population
# pop = [generate_population(N_WEIGHTS,PRECISION) for _ in range(8)]
# pop_pool = generate_pool(PRECISION)
# print(pop)
# print(pop_pool)

# # the fitness function evaluates how well a gene does. in this case we insert the weights
# # into a player into npare games, which i've changed it to return the win percentage
# pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
# fitness= lambda name, w:jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(name,w),1)

#call the genetic algorithm
# best_fit_weights = genetic_algorithm_custom(pop,fitness,pop_pool,ngen=N_GENERATION,pmut=PMUT)

# print(best_fit_weights)


# Tornament

In [51]:
def incorpora(tabela,tx):
    for jog in tx:
        if jog not in tabela:
            tabela[jog]=tx[jog]
        else:
            tabela[jog]+=tx[jog]
    
def torneio(n,jogadores,jogo_id=0):
    jogo=Kalah(jogo_id)  # jogo gerado ao calha entre os "justos"
    tabela={}
    for i in range(len(jogadores)-1):
        jog1=jogadores[i]
        for j in range(i+1,len(jogadores)):
            jog2=jogadores[j]
            _,_,_,tabelaX = jogaNpares(jogo,n,jog1,jog2)
            incorpora(tabela,tabelaX)
    return tabela

#n - number of games
#jogadores - list of players
def torneio_GA(n,main_character,jogadores,jogo_id=0):
    jogo=Kalah(jogo_id)  # jogo gerado ao calha entre os "justos"
    jogadores.append(main_character)
    tabela={}
    print(' ', end='', flush=True)
    # for _ in tqdm(range(len(jogadores)-1), desc= "torneio_GA in progress %d"%jog2.nome,leave=False):
    for i in tqdm(range(len(jogadores)-1), desc= "torneio_GA in progress %d"%main_character.nome,leave=False):
        jog1=jogadores[i]
        for j in range(i+1,len(jogadores)):
            jog2=jogadores[j]
            _,_,_,tabelaX = jogaNpares(jogo,n,jog1,jog2) # tabelaX = {'Pebbles': 24, 'Pebbles_BestCapture': 36}
            incorpora(tabela,tabelaX)
    return calculate_win(tabela,main_character)

def calculate_win(table,main_character):
    all_score = sum(table.values())
    print(table)
    for each_player in table:
        if each_player == main_character.nome:
            return table[each_player]/all_score #hopefully returns a win percentage
            


# KEVDEV GA - tournament style

In [52]:
#backup - kev dev
from searchPlus import genetic_algorithm_custom
from kevdiff import genetic_algorithm_custom as ga_kev
import random

# one generation of this takes 1m30s/90s

# for steps in 0.01, use 2. 
DEPTH = depth  #3 rn
# PROTAG STATS
HEURISTICS = h_list
N_WEIGHTS = len(HEURISTICS)
N_GAMES = 30
N_GENERATION = 50
PMUT = 0.1
N_POPULATION = 8
# ENEMIES
ENEMY_LIST = [toe1,pebs,pebs_bc,pebs_bd]

# e.g. [0.7, 0.12, 0.12, 0.06]
def generate_population(n):
    return normalize([random.random() for _ in range(n)])

# generates a pool of potential changes from 0-1
# a list that goes [0.0, 0.01 ... 1] essentially. based on precision
# def generate_pool(precision):
#     return [round(x*pow(10,-precision),precision) for x in range(pow(10,precision))]

prev_good_weights_for_defense = [[0.53, 0.01, 0.03, 0.03]]
pop_rest = [generate_population(N_WEIGHTS) for _ in range(N_POPULATION-len(prev_good_weights_for_defense))]
pop = prev_good_weights_for_defense + pop_rest

#mid testing for capture focused
# pop_gen17 = [(296, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5922330097087378), (181, [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], 0.5873015873015873), (326, [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], 0.5849056603773585), (281, [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], 0.5754716981132075), (247, [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], 0.573208722741433), (340, [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], 0.5728155339805825), (218, [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], 0.5705128205128205), (328, [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], 0.5682539682539682), (304, [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], 0.565359477124183), (216, [0.5244964022687261, 0.0024821347540840543, 0.16553140370103708, 0.23074434922059944, 0.07674571005555327], 0.565359477124183), (183, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.565359477124183), (191, [0.4220340143246191, 0.0016217516845741953, 0.3222589517171829, 0.21351662545668781, 0.040568656816935986], 0.5638629283489096), (361, [0.5321287927976377, 0.0024422935796346032, 0.1628744304964742, 0.22704063174297384, 0.0755138513832796], 0.5631067961165048), (244, [0.503236853071467, 0.003219031462752779, 0.2098999632357117, 0.23835596329214934, 0.04528818893791916], 0.5566037735849056), (343, [0.5278617454660057, 0.0012144229189132074, 0.16479734850271316, 0.22972110477729635, 0.07640537833507148], 0.5533980582524272), (286, [0.47551148641798996, 0.0012635841717298889, 0.2510873362804695, 0.204215420729435, 0.06792217240037564], 0.5533980582524272), (337, [0.4824081948782528, 0.00253728743925837, 0.2879892244193175, 0.19081079683976004, 0.036254496423411206], 0.5512820512820513), (119, [0.4567526386179326, 0.0014511037277271798, 0.28834942524079316, 0.04219666311629308, 0.21125016929725385], 0.5471698113207547), (321, [0.4240589846181617, 0.0016295330454913263, 0.3238051891812408, 0.21050907747006667, 0.03999721568503944], 0.545751633986928), (268, [0.4589515295334259, 0.002413914115632853, 0.24539703161033458, 0.2200490707654506, 0.07318845397515614], 0.545751633986928), (145, [0.42096729007497213, 0.0016247448742976692, 0.3228537296919815, 0.21391070290713027, 0.04064353245161829], 0.545751633986928), (205, [0.5242839050426428, 0.0027575380787338167, 0.17980785464343735, 0.21998391808195197, 0.07316678415323409], 0.5451713395638629), (305, [0.4641343575158191, 0.0012333515476795063, 0.24507979898187618, 0.21728377651802178, 0.07226871543660336], 0.5416666666666666), (239, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5416666666666666)]
# pop = []
# for ind in pop_gen17:
#     pop.append(ind[1])
# # pop_pool = generate_pool(PRECISION)
# print(pop)

# print(pop_pool)

# the fitness function evaluates how well a gene does. in this case we insert the weights
# into a player into npare games, which i've changed it to return the win percentage
# pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
# fitness= lambda name, w:jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(name,w),1)

def fitness (tuple): #tuple(name:int, weights:list[float])
    name = tuple[0]
    weights = tuple[1]
    pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
    anime_protag = pebs_generator(tuple[0],tuple[1])
    
    return (name,weights,torneio_GA(N_GAMES,anime_protag,ENEMY_LIST)) # torneio_GA(n,main_character,jogadores,jogo_id=0):

#call the genetic algorithm
print(pop)

###TESTING OF THE NEW HEURISTIC WORKS### N_GAMES = 30, against 5 enemies
# anime_protag = (123,[0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898])
# print(fitness(anime_protag))

best_fit_weights = ga_kev(pop,fitness,[],ngen=N_GENERATION,pmut=PMUT)
print(best_fit_weights)

[[0.53, 0.01, 0.03, 0.03], [0.20840402189052518, 0.3454044719257833, 0.09890645144131613, 0.34728505474237537], [0.021317912726552653, 0.24235480292362446, 0.36028116711425384, 0.3760461172355691], [0.16871614380163089, 0.41252581919030723, 0.18082274856111843, 0.23793528844694353], [0.5663879499383544, 0.25615192445654533, 0.0861899653216611, 0.09127016028343918], [0.36108588815031567, 0.3214300819868612, 0.26338536438276267, 0.054098665480060565], [0.2724751284279748, 0.05607157263302363, 0.1571121899830583, 0.5143411089559433], [0.19151817042267527, 0.37717748923109934, 0.19864409649267933, 0.2326602438535462]]


overall gen:   0%|          | 0/50 [00:00<?, ?it/s]

eval pop:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [60]:
list1 = [[0.74, 0.01, 0.09, 0.06, 0.15], [0.53, 0.01, 0.03, 0.03, 0.04], [0.8386724534373374, 0.0047944633947019175, 0.0, 0.15653308316796063, 0.0]]
list2 = [[1],[1,2]]
list3 = shuffled(list1+list2)
print(list3)
print(random.uniform(-0.5,1))
c = 0.3
d =  c+random.uniform(-0.5,0.5)
print(d)

[[1], [1, 2], [0.53, 0.01, 0.03, 0.03, 0.04], [0.8386724534373374, 0.0047944633947019175, 0.0, 0.15653308316796063, 0.0], [0.74, 0.01, 0.09, 0.06, 0.15]]
0.8170545006393581
0.46278202968124343


# KEVDEV GA

In [37]:
#backup - kev dev
from searchPlus import genetic_algorithm_custom
from kevdiff import genetic_algorithm_custom as ga_kev
import random

# for steps in 0.01, use 2. 
DEPTH = depth  #6 rn
# PRECISION = 2
HEURISTICS = alt_h_list
N_WEIGHTS = 5
TRAINING_ENEMY = pebs4
N_GAMES = 50
N_GENERATION = 100
PMUT = 0.1
N_POPULATION = 24
# generates n weights that adds up to 1, precision see above
# e.g. [0.7, 0.12, 0.12, 0.06]
def generate_population(n):
    return normalize([random.random() for _ in range(n)])

# generates a pool of potential changes from 0-1
# a list that goes [0.0, 0.01 ... 1] essentially. based on precision
# def generate_pool(precision):
#     return [round(x*pow(10,-precision),precision) for x in range(pow(10,precision))]
    
# generates 8 "gene" as an initial population
# pop = [generate_population(N_WEIGHTS) for _ in range(N_POPULATION)]

pop_gen17 = [(296, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5922330097087378), (181, [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], 0.5873015873015873), (326, [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], 0.5849056603773585), (281, [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], 0.5754716981132075), (247, [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], 0.573208722741433), (340, [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], 0.5728155339805825), (218, [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], 0.5705128205128205), (328, [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], 0.5682539682539682), (304, [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], 0.565359477124183), (216, [0.5244964022687261, 0.0024821347540840543, 0.16553140370103708, 0.23074434922059944, 0.07674571005555327], 0.565359477124183), (183, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.565359477124183), (191, [0.4220340143246191, 0.0016217516845741953, 0.3222589517171829, 0.21351662545668781, 0.040568656816935986], 0.5638629283489096), (361, [0.5321287927976377, 0.0024422935796346032, 0.1628744304964742, 0.22704063174297384, 0.0755138513832796], 0.5631067961165048), (244, [0.503236853071467, 0.003219031462752779, 0.2098999632357117, 0.23835596329214934, 0.04528818893791916], 0.5566037735849056), (343, [0.5278617454660057, 0.0012144229189132074, 0.16479734850271316, 0.22972110477729635, 0.07640537833507148], 0.5533980582524272), (286, [0.47551148641798996, 0.0012635841717298889, 0.2510873362804695, 0.204215420729435, 0.06792217240037564], 0.5533980582524272), (337, [0.4824081948782528, 0.00253728743925837, 0.2879892244193175, 0.19081079683976004, 0.036254496423411206], 0.5512820512820513), (119, [0.4567526386179326, 0.0014511037277271798, 0.28834942524079316, 0.04219666311629308, 0.21125016929725385], 0.5471698113207547), (321, [0.4240589846181617, 0.0016295330454913263, 0.3238051891812408, 0.21050907747006667, 0.03999721568503944], 0.545751633986928), (268, [0.4589515295334259, 0.002413914115632853, 0.24539703161033458, 0.2200490707654506, 0.07318845397515614], 0.545751633986928), (145, [0.42096729007497213, 0.0016247448742976692, 0.3228537296919815, 0.21391070290713027, 0.04064353245161829], 0.545751633986928), (205, [0.5242839050426428, 0.0027575380787338167, 0.17980785464343735, 0.21998391808195197, 0.07316678415323409], 0.5451713395638629), (305, [0.4641343575158191, 0.0012333515476795063, 0.24507979898187618, 0.21728377651802178, 0.07226871543660336], 0.5416666666666666), (239, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5416666666666666)]
pop = []
for ind in pop_gen17:
    pop.append(ind[1])
# pop_pool = generate_pool(PRECISION)
print(pop)
# print(pop_pool)

# the fitness function evaluates how well a gene does. in this case we insert the weights
# into a player into npare games, which i've changed it to return the win percentage
# pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
# fitness= lambda name, w:jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(name,w),1)

def fitness (tuple): #tuple(name:int, weights:list[float], fitness:float)
    name = tuple[0]
    weights = tuple[1]
    pebs_generator = lambda name,w:JogadorLinearPond(name,DEPTH,w,HEURISTICS,100)
    return (name,weights,jogaNpares_GA(jogo,N_GAMES,TRAINING_ENEMY,pebs_generator(tuple[0],tuple[1]),1))

###TESTING OF THE NEW HEURISTIC WORKS### N_GAMES = 30, against 5 enemies
anime_protag = (123,[0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898])
print(fitness(anime_protag))

#call the genetic algorithm
# best_fit_weights = ga_kev(pop,fitness,[],ngen=N_GENERATION,pmut=PMUT)
# #pop_pool ignored

# print(best_fit_weights)

[[0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], [0.5244964022687261, 0.002482134754

(123, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.43119266055045874)


In [ ]:
pop_gen17 = [(296, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5922330097087378), (181, [0.5060064283744496, 0.0023946325203097678, 0.27945500045902055, 0.17827186795515407, 0.03387207069106592], 0.5873015873015873), (326, [0.5087305384943449, 0.0030431687499736727, 0.17085225489545827, 0.2381614091784416, 0.07921262868178149], 0.5849056603773585), (281, [0.5492302031235177, 0.002888746315759254, 0.15673364697615585, 0.2184806179605766, 0.07266678562399057], 0.5754716981132075), (247, [0.41752397451100454, 0.0012991470449279108, 0.29854278707734244, 0.21209212302177424, 0.07054196834495088], 0.573208722741433), (340, [0.4293397173643812, 0.0012727933638494399, 0.2924867355859959, 0.20778975540973707, 0.06911099827603649], 0.5728155339805825), (218, [0.43822936301080534, 0.001363572912484629, 0.31334779174201893, 0.20761242699559582, 0.0394468453390953], 0.5705128205128205), (328, [0.5141678293348594, 0.002536050034560504, 0.16912696677170333, 0.2357564245263881, 0.07841272933248868], 0.5682539682539682), (304, [0.5115214949486129, 0.0011768297898419075, 0.23384833697495405, 0.19019452446170035, 0.06325881382489079], 0.565359477124183), (216, [0.5244964022687261, 0.0024821347540840543, 0.16553140370103708, 0.23074434922059944, 0.07674571005555327], 0.565359477124183), (183, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.565359477124183), (191, [0.4220340143246191, 0.0016217516845741953, 0.3222589517171829, 0.21351662545668781, 0.040568656816935986], 0.5638629283489096), (361, [0.5321287927976377, 0.0024422935796346032, 0.1628744304964742, 0.22704063174297384, 0.0755138513832796], 0.5631067961165048), (244, [0.503236853071467, 0.003219031462752779, 0.2098999632357117, 0.23835596329214934, 0.04528818893791916], 0.5566037735849056), (343, [0.5278617454660057, 0.0012144229189132074, 0.16479734850271316, 0.22972110477729635, 0.07640537833507148], 0.5533980582524272), (286, [0.47551148641798996, 0.0012635841717298889, 0.2510873362804695, 0.204215420729435, 0.06792217240037564], 0.5533980582524272), (337, [0.4824081948782528, 0.00253728743925837, 0.2879892244193175, 0.19081079683976004, 0.036254496423411206], 0.5512820512820513), (119, [0.4567526386179326, 0.0014511037277271798, 0.28834942524079316, 0.04219666311629308, 0.21125016929725385], 0.5471698113207547), (321, [0.4240589846181617, 0.0016295330454913263, 0.3238051891812408, 0.21050907747006667, 0.03999721568503944], 0.545751633986928), (268, [0.4589515295334259, 0.002413914115632853, 0.24539703161033458, 0.2200490707654506, 0.07318845397515614], 0.545751633986928), (145, [0.42096729007497213, 0.0016247448742976692, 0.3228537296919815, 0.21391070290713027, 0.04064353245161829], 0.545751633986928), (205, [0.5242839050426428, 0.0027575380787338167, 0.17980785464343735, 0.21998391808195197, 0.07316678415323409], 0.5451713395638629), (305, [0.4641343575158191, 0.0012333515476795063, 0.24507979898187618, 0.21728377651802178, 0.07226871543660336], 0.5416666666666666), (239, [0.5408094183539636, 0.00284445612406821, 0.15969595598418704, 0.22260996169213218, 0.07404020784564898], 0.5416666666666666)]
pop = []
for ind in pop_gen17:
    pop.append(ind[1])
# pop_pool = generate_pool(PRECISION)
print(pop)

In [ ]:
list1 = [[1, 2, 3, 4, 5],0.1]
list2 = [['a', 'b', 'c', 'd', 'e'],0.4]
list3 = [[0, 9, 8, 7, 6],0.2]

def wah(*lists):
    listee = [x[0] for x in [*lists]]
    print(listee)
    return [list(value) for value in zip(*listee)]
babha = wah(list1,list2,list3)
print(babha)


listx = []
listx.append(list1)
listx.append(list2)
print(listx)

In [64]:
# # from tqdm import tqdm
# from random import sample
# # for i in tqdm(range(10000000)):
# #     pass

# x = sorted(sample(range(1, 100), 5 - 1))
# print(x)


w = [0.2586993457142682, 0.010680145858320323, 0.6757607019062054, 0.00742569201034956, 0.04743411451085642]
pebs_bd = JogadorLinearPond('Pebbles_BestDefense',depth,w,full_h_list,100)


# w_best = [0.4132746969431487, 0.004000018652774955, 0.09436108536199529, 0.0566795623905725, 0.029663753065053162, 0.4020208835864555]
# pebs_final = JogadorLinearPond('Peb_Final',depth,w,full_h_list,100)

jogs = [toe1,pebs,pebs_bd,pebreaper]
# jogs = [toe1,pebs,pebs2,pebs2b,pebs3b,pebs4b]
torneio(30,jogs,0)

{'chapiteau': 92,
 'Pebbles': 347,
 'Pebbles_BestDefense': 295,
 'PebbleReaper': 330}

In [32]:
random.randrange(0,10)

4

In [ ]:

'''
      _   _  _  _  _  _
   _  12 11 10  9  8  7     N
   13                    6 
       0  1  2  3  4  5     S

'''
board = [5, 14, 6, 14, 14, 0, 0, 14, 1, 0, 14, 14, 14, 0]

south_max_capture = 0
south_empties = [i for i in range(6) if board[i]==0]
south_move_final_index = [(i+board[i])%14 if i+board[i]<13 else (i+board[i])%13 for i in range(6) if board[i] != 0 and board[i]<=13]
south_possible_captures = [value for value in south_empties if value in south_move_final_index]

if len(south_possible_captures) != 0:
    for i in south_possible_captures:
        if 1+board[i+7] > south_max_capture:
            south_max_capture = 1+board[i+7]

north_max_capture = 0
north_empties = [i for i in range(7,13) if board[i]==0]
north_move_final_index = [(i+board[i])%14 if i+board[i]-7<13 else ((i+board[i])%14)+1 for i in range(7,13) if board[i] != 0 and board[i]<=13]
north_possible_captures = [value for value in north_empties if value in north_move_final_index]

if len(north_possible_captures) != 0:
    for i in north_possible_captures:
        print("1+board[i-7]",1+board[i-7], "i",i)
        if 1+board[i-7] > north_max_capture:
            north_max_capture = 1+board[i-7]


# north_move_final_index = [(i+board[i])%14 if i+board[i]-7<13 else (i+(board[i]%13))%13 for i in range(7,13)]
print(south_empties)
print(south_move_final_index)
print(south_possible_captures)
print(south_max_capture)

print(north_empties)
print(north_move_final_index)
print(north_possible_captures)
print(north_max_capture)




#0,1,2,3,4,6